In [4]:
!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://pypicloud.optoro.io/pypi
airflow 1.8.0 has requirement jinja2<2.9.0,>=2.7.3, but you'll have jinja2 2.10 which is incompatible.
bleach 2.1.3 has requirement html5lib!=1.0b1,!=1.0b2,!=1.0b3,!=1.0b4,!=1.0b5,!=1.0b6,!=1.0b7,!=1.0b8,>=0.99999999pre, but you'll have html5lib 1.0b8 which is incompatible.
smart-open 1.3.2 has requirement requests==2.8.1, but you'll have requests 2.9.1 which is incompatible.


In [28]:
!pip install pygeocoder

Looking in indexes: https://pypi.org/simple, https://pypicloud.optoro.io/pypi
  Running setup.py bdist_wheel for pygeocoder ... done
  Stored in directory: /Users/tterry/Library/Caches/pip/wheels/7c/4c/00/d05c66c4af5411c554c91b8079732c8a0359c2226fb8c01031
Successfully built pygeocoder
smart-open 1.3.2 has requirement requests==2.8.1, but you'll have requests 2.9.1 which is incompatible.
airflow 1.8.0 has requirement jinja2<2.9.0,>=2.7.3, but you'll have jinja2 2.10 which is incompatible.
bleach 2.1.3 has requirement html5lib!=1.0b1,!=1.0b2,!=1.0b3,!=1.0b4,!=1.0b5,!=1.0b6,!=1.0b7,!=1.0b8,>=0.99999999pre, but you'll have html5lib 1.0b8 which is incompatible.


In [6]:
!ls

DCRA Determination Letter for FOIA Request #2018-FOIA-03417 (Code for DC).pdf
Untitled.ipynb
Untitled.py
Untitled.txt
Vacant Buildings  as of February 7 2018.pdf
Vacant_Exempt.pdf


In [18]:
from tabula import read_pdf
from pygeocoder import Geocoder

In [5]:
page_one_data = read_pdf('../Vacant Buildings  as of February 7 2018.pdf')

In [6]:
# manually confirmed that page 1 has 42 rows plus header
# another one I checked has 43
page_one_data.shape

(42, 6)

In [7]:
# the file is 23 pages
42*23

966

In [8]:
all_data = read_pdf('../Vacant Buildings  as of February 7 2018.pdf', pages='all')

In [9]:
# 981 seems about right
all_data.shape

(981, 6)

In [10]:
all_data.head().T

,0,1,2,3,4
WARD,1,1,1,1,1
STREET NUMBER,2002,2002,2001,982,2014
STREET NAME,11TH,11TH,11TH,FLORIDA,9TH
STREET TYPE,ST,ST,ST,AVE,ST
QUADRANT,NW,NW,NW,NW,NW
SQUARE SUFFIX LOT,0304 0027,0304 0825,0332 0080,0357 0098,0360 0078


In [11]:
all_data.columns = [c.replace(' ', '_').lower() for c in all_data.columns]

In [12]:
all_data.ward.value_counts()

7       273
8       171
5       165
4       110
1        91
6        86
2        35
3        28
WARD     22
Name: ward, dtype: int64

In [16]:
all_data = all_data[all_data.ward != 'WARD']

In [17]:
all_data.shape

(959, 6)

# Let's Geocode these Addresses

Good intro at https://chrisalbon.com/python/data_wrangling/geocoding_and_reverse_geocoding/

In [26]:
result = Geocoder.geocode("1600 amphiteather parkway, mountain view, CA 94043")

In [27]:
result.latitude, result.longitude, result.valid_address

(37.4214614, -122.0827013, True)

In [32]:
all_data.dtypes

ward                 object
street_number        object
street_name          object
street_type          object
quadrant             object
square_suffix_lot    object
dtype: object

In [33]:
' '.join(['a', 'b'])

'a b'

In [37]:
all_data['address_full'] = all_data.apply(lambda x: ' '.join([x['street_number'], x['street_type'], x['quadrant'], 'Washington, DC'], axis=1))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "pandas/_libs/index.pyx", line 154, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5126)
  File "pandas/_libs/hashtable_class_helper.pxi", line 759, in pandas._libs.hashtable.Int64HashTable.get_item (pandas/_libs/hashtable.c:14010)
TypeError: an integer is required

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tterry/.pyenv/versions/3.5.1/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-37-fca46aea3cae>", line 1, in <module>
    all_data['address_full'] = all_data.apply(lambda x: ' '.join([x['street_number'], x['street_type'], x['quadrant'], 'Washington, DC'], axis=1))
  File "/Users/tterry/.pyenv/versions/3.5.1/lib/python3.5/site-packages/pandas/core/frame.py", line 4262, in apply
    ignore_failures=ignore_failures)
  File "/Users/tterry/.py

TypeError: Can't convert 'list' object to str implicitly

In [44]:
all_data['address_full'] = all_data.apply(lambda x: x['street_number'] + ' ' + x['street_name'] + ' ' + x['street_type'] + ' ' + x['quadrant'] + ' ' + 'Washington, DC', axis=1)

In [48]:
all_data['geocode_object'] = all_data.address_full.apply(Geocoder.geocode)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/tterry/.pyenv/versions/3.5.1/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-48-30b3225b8516>", line 1, in <module>
    all_data['geocode_object'] = all_data.address_full.apply(Geocoder.geocode)
  File "/Users/tterry/.pyenv/versions/3.5.1/lib/python3.5/site-packages/pandas/core/series.py", line 2355, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/src/inference.pyx", line 1574, in pandas._libs.lib.map_infer (pandas/_libs/lib.c:66645)
  File "/Users/tterry/.pyenv/versions/3.5.1/lib/python3.5/site-packages/pygeocoder.py", line 129, in geocode
    return GeocoderResult(Geocoder.get_data(params=params))
  File "/Users/tterry/.pyenv/versions/3.5.1/lib/python3.5/site-packages/pygeocoder.py", line 212, in get_data
    raise GeocoderError(response_json['status'], response.url)
pygeolib.Geo

GeocoderError: Error OVER_QUERY_LIMIT
Query: https://maps.google.com/maps/api/geocode/json?bounds=&components=&sensor=false&region=&address=2412+18TH+ST+NW+Washington%2C+DC&language=

In [47]:
Geocoder.geocode('2002 11TH ST NW Washington, DC').valid_address

True